In [16]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time
import torch.nn.functional as F

/home/octav/gitrepos/tum-thesis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
model_type = "pointwise"
sys.argv = ["", "-cuda", "-model_type", model_type, "-one_batch_train"]
args = get_args()
args.n_epochs = 1000 
args.init_model = f"{model_type}10-onebatch"
args.output_dir = "mol_opt/output_{}".format(args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}".format(args.init_model)
args.init_model = "{}10-onebatch-euler-valency-scale2".format(model_type)
# args.init_model = "{}10-dev2".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)
args.batch_size = 50 

args.annealing_rate = 0.0002
args.connectivity_lambda = 0
args.valency_lambda = 1
args.euler_lambda = 1
args.connectivity = False 
args.valency = True 
args.euler_characteristic_penalty = True
args.scale_lambdas = True
args.conn_penalty_function = "logdet" 
args

Namespace(N_transformer=6, agg_func='sum', annealing_rate=0.0002, batch_norm=False, batch_size=50, conn_penalty_function='logdet', connectivity=False, connectivity_hard=False, connectivity_lambda=0, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=True, euler_lambda=1, ffn_activation='LeakyReLU', init_decoder_model='pointwise10-onebatch-euler-valency-scale2_decode', init_model='pointwise10-onebatch-euler-valency-scale2', linear_out=False, model_type='pointwise', n_epochs=1000, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=250, n_labels=1, n_layers=5, one_batch_train=True, ot_solver='emd', output_dir='mol_opt/output_pointwise10-onebatch', pc_hidden=100, pred_hidden=150, scale_lambdas=True, sinkhorn_entropy=0.1, sinkhorn_max_it=10000, task='qed', tb_logs_dir='mol_opt/logs_pointwise10-onebatch', valency=True, valency_hard=False, valency_lambda=1)

In [18]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, same_number_atoms = True)
# val_data_loader = get_loader("iclr19-graph2graph/data/qed", "valid", 36, False)

In [19]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = None)

No model pointwise10-onebatch-euler-valency-scale2 found in mol_opt/output_pointwise10-onebatch! Starting from scratch.
Epoch: 1
Epoch 1, train
 train_fgw:3.6789285
 train_conn_penalty:0.0000000
 train_val_penalty:550.5089062
 train_euler_penalty:161.3134766
 train_total:715.5013281
 train_similarity:0.0000000
 train_penlog:-100.0000000
Epoch duration: 1.2415337562561035
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_1
Epoch: 2
Epoch 2, train
 train_fgw:3.0796664
 train_conn_penalty:0.0000000
 train_val_penalty:406.0726172
 train_euler_penalty:137.3011426
 train_total:546.5620703
 train_similarity:0.0004651
 train_penlog:-21.0847836
Epoch duration: 1.1578047275543213
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_2
Epoch: 3
Epoch 3, train
 train_fgw:3.4028571
 train_conn_penalty:0.0000000
 train_val_penalty:21.3777441
 train_euler_penalty:20.4819531
 train_total:45.2792969
 train_similarity:0.0004651
 train_penlo

Epoch 25, train
 train_fgw:4.5007523
 train_conn_penalty:0.0000000
 train_val_penalty:21.5800000
 train_euler_penalty:20.5800000
 train_total:46.8635840
 train_similarity:0.0000000
 train_penlog:4.8874377
Epoch duration: 1.19063138961792
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_25
Epoch: 26
Epoch 26, train
 train_fgw:5.0249478
 train_conn_penalty:0.0000000
 train_val_penalty:21.5800000
 train_euler_penalty:20.5800000
 train_total:47.3962549
 train_similarity:0.0000000
 train_penlog:4.8874377
Epoch duration: 1.0751583576202393
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_26
Epoch: 27
Epoch 27, train
 train_fgw:5.4982751
 train_conn_penalty:0.0000000
 train_val_penalty:21.5800000
 train_euler_penalty:20.5800000
 train_total:47.8780566
 train_similarity:0.0000000
 train_penlog:4.8874377
Epoch duration: 1.0686750411987305
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_27
E

Epoch 50, train
 train_fgw:2.7671463
 train_conn_penalty:0.0000000
 train_val_penalty:21.4946509
 train_euler_penalty:20.5544653
 train_total:45.2303223
 train_similarity:0.0000000
 train_penlog:4.8874377
Epoch duration: 1.1402738094329834
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_50
Epoch: 51
Epoch 51, train
 train_fgw:3.2819397
 train_conn_penalty:0.0000000
 train_val_penalty:21.5799487
 train_euler_penalty:20.5799756
 train_total:45.8655371
 train_similarity:0.0000000
 train_penlog:4.8874377
Epoch duration: 1.0844857692718506
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_51
Epoch: 52
Epoch 52, train
 train_fgw:3.7495212
 train_conn_penalty:0.0000000
 train_val_penalty:21.5800000
 train_euler_penalty:20.5800000
 train_total:46.3417090
 train_similarity:0.0000000
 train_penlog:4.8874377
Epoch duration: 1.0797224044799805
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_52

Epoch 75, train
 train_fgw:3.6697070
 train_conn_penalty:0.0000000
 train_val_penalty:21.5800000
 train_euler_penalty:20.5800000
 train_total:46.4582520
 train_similarity:0.0000000
 train_penlog:4.8874377
Epoch duration: 1.0767326354980469
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_75
Epoch: 76
Epoch 76, train
 train_fgw:3.5247208
 train_conn_penalty:0.0000000
 train_val_penalty:21.5800000
 train_euler_penalty:20.5800000
 train_total:46.3218262
 train_similarity:0.0000000
 train_penlog:4.8874377
Epoch duration: 1.1154730319976807
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_76
Epoch: 77
Epoch 77, train
 train_fgw:3.3756271
 train_conn_penalty:0.0000000
 train_val_penalty:21.5800000
 train_euler_penalty:20.5800000
 train_total:46.1812891
 train_similarity:0.0000000
 train_penlog:4.8874377
Epoch duration: 1.1614575386047363
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_77

Epoch 100, train
 train_fgw:8.9955273
 train_conn_penalty:0.0000000
 train_val_penalty:21.5800000
 train_euler_penalty:20.5800000
 train_total:51.9985303
 train_similarity:0.0000000
 train_penlog:4.8874377
Epoch duration: 1.120936393737793
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_100
Epoch: 101
Epoch 101, train
 train_fgw:9.8587518
 train_conn_penalty:0.0000000
 train_val_penalty:21.5800000
 train_euler_penalty:20.5800000
 train_total:52.8703516
 train_similarity:0.0000000
 train_penlog:4.8874377
Epoch duration: 1.0854005813598633
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_101
Epoch: 102
Epoch 102, train
 train_fgw:10.6075916
 train_conn_penalty:0.0000000
 train_val_penalty:21.5800000
 train_euler_penalty:20.5800000
 train_total:53.6277979
 train_similarity:0.0000000
 train_penlog:4.8874377
Epoch duration: 1.0729637145996094
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-sc

...

Epoch 900, train
 train_fgw:0.9611518
 train_conn_penalty:0.0000000
 train_val_penalty:0.2059366
 train_euler_penalty:0.0094121
 train_total:1.2189148
 train_similarity:0.0354404
 train_penlog:-15.6926091
Epoch duration: 1.1264922618865967
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_900
Epoch: 901
Epoch 901, train
 train_fgw:0.9790991
 train_conn_penalty:0.0000000
 train_val_penalty:0.2592014
 train_euler_penalty:0.3905345
 train_total:1.7569601
 train_similarity:0.0317937
 train_penlog:-13.4743187
Epoch duration: 1.2451415061950684
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_901
Epoch: 902
Epoch 902, train
 train_fgw:0.9626135
 train_conn_penalty:0.0000000
 train_val_penalty:0.3743743
 train_euler_penalty:0.0427844
 train_total:1.4621339
 train_similarity:0.0343424
 train_penlog:-13.6265642
Epoch duration: 1.4366793632507324
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale

Epoch 925, train
 train_fgw:0.8065650
 train_conn_penalty:0.0000000
 train_val_penalty:0.3221915
 train_euler_penalty:0.0560514
 train_total:1.2615746
 train_similarity:0.0254243
 train_penlog:-30.4795852
Epoch duration: 1.1078503131866455
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_925
Epoch: 926
Epoch 926, train
 train_fgw:0.8003511
 train_conn_penalty:0.0000000
 train_val_penalty:0.4585738
 train_euler_penalty:1.2236106
 train_total:2.8243491
 train_similarity:0.0269444
 train_penlog:-26.5047891
Epoch duration: 1.11393141746521
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_926
Epoch: 927
Epoch 927, train
 train_fgw:0.8101830
 train_conn_penalty:0.0000000
 train_val_penalty:0.2901162
 train_euler_penalty:0.0107611
 train_total:1.1722698
 train_similarity:0.0285995
 train_penlog:-20.3212024
Epoch duration: 1.098062515258789
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_9

Epoch 950, train
 train_fgw:0.8389480
 train_conn_penalty:0.0000000
 train_val_penalty:0.1811293
 train_euler_penalty:0.4702918
 train_total:1.6265063
 train_similarity:0.0313958
 train_penlog:-21.9827573
Epoch duration: 1.1729457378387451
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_950
Epoch: 951
Epoch 951, train
 train_fgw:0.8604694
 train_conn_penalty:0.0000000
 train_val_penalty:0.2159061
 train_euler_penalty:0.0129239
 train_total:1.1371767
 train_similarity:0.0310475
 train_penlog:-18.0362344
Epoch duration: 1.115910291671753
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_951
Epoch: 952
Epoch 952, train
 train_fgw:0.8723129
 train_conn_penalty:0.0000000
 train_val_penalty:0.2853091
 train_euler_penalty:0.1183957
 train_total:1.3605811
 train_similarity:0.0320707
 train_penlog:-7.5755303
Epoch duration: 1.166515588760376
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_9

Epoch 975, train
 train_fgw:0.8983702
 train_conn_penalty:0.0000000
 train_val_penalty:0.1500759
 train_euler_penalty:0.0304235
 train_total:1.1176850
 train_similarity:0.0261973
 train_penlog:-28.8501822
Epoch duration: 1.1974799633026123
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_975
Epoch: 976
Epoch 976, train
 train_fgw:0.9178017
 train_conn_penalty:0.0000000
 train_val_penalty:0.2971329
 train_euler_penalty:0.0583023
 train_total:1.3497577
 train_similarity:0.0228999
 train_penlog:-22.8214399
Epoch duration: 1.134573221206665
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_976
Epoch: 977
Epoch 977, train
 train_fgw:0.9422737
 train_conn_penalty:0.0000000
 train_val_penalty:0.3129094
 train_euler_penalty:0.1963747
 train_total:1.5613240
 train_similarity:0.0231776
 train_penlog:-19.0315665
Epoch duration: 1.219597578048706
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_

Epoch 1000, train
 train_fgw:0.7926528
 train_conn_penalty:0.0000000
 train_val_penalty:0.2835259
 train_euler_penalty:0.0260427
 train_total:1.1706775
 train_similarity:0.0212460
 train_penlog:-49.5973618
Epoch duration: 1.2466545104980469
mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch-euler-valency-scale2_1000


In [41]:
for i in train_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break
    
x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

from mol_opt.ot_utils import FGW 
fgw_loss = FGW(alpha = 0.5)
fgw_loss(*pred_pack, tau = 1)

tensor(0.8065, device='cuda:0', grad_fn=<AddBackward0>)

In [45]:
molopt.GCN(X)

(tensor([[ 0.3615,  0.0986, -0.0310,  ..., -0.2754, -0.5545,  0.3003],
         [ 0.1664, -0.3153, -0.1335,  ..., -0.0280, -0.4702,  0.0394],
         [ 0.1754, -0.2700,  0.0417,  ..., -0.3650, -0.0990,  0.1231],
         ...,
         [ 0.6155, -0.7656,  0.2039,  ..., -0.1572, -0.1240,  0.3885],
         [ 0.6516, -0.9200,  0.2562,  ..., -0.0776, -0.1325,  0.5674],
         [ 0.7312, -1.2065,  0.4469,  ...,  0.1661,  0.0923,  0.8543]],
        device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[1.3345],
         [1.4662],
         [2.0025],
         [0.9231],
         [2.1800],
         [1.9749]], device='cuda:0', grad_fn=<AddmmBackward>))

In [48]:
list(molopt.GCN.parameters())

[Parameter containing:
 tensor([[-0.0271, -0.0222,  0.1718,  ..., -0.0321, -0.0936, -0.1356],
         [-0.0505, -0.0601, -0.0955,  ...,  0.0704, -0.0691,  0.0460],
         [-0.0407, -0.1047, -0.1167,  ..., -0.0943,  0.0353,  0.0042],
         ...,
         [-0.0035,  0.1513, -0.0326,  ...,  0.0094,  0.0024,  0.0582],
         [ 0.0236, -0.0382,  0.0031,  ...,  0.0195, -0.0009,  0.0244],
         [-0.1007,  0.0423, -0.1154,  ..., -0.0303, -0.0439, -0.0662]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[-0.2061, -0.0520, -0.0368,  ..., -0.1384, -0.0814, -0.1330],
         [ 0.0212, -0.0873,  0.0341,  ..., -0.0692,  0.0360,  0.0197],
         [-0.1153, -0.0140, -0.1584,  ..., -0.0471, -0.0504,  0.0007],
         ...,
         [-0.1464, -0.0601, -0.0457,  ...,  0.0594, -0.0490, -0.0346],
         [ 0.0554, -0.0554, -0.0424,  ..., -0.0040, -0.0070, -0.0566],
         [-0.1206, -0.0652, -0.0773,  ..., -0.0139, -0.0578, -0.1133]],
        device='cuda:0', r

In [ ]:
tau = 1
symbols_nll, charges_nll, bonds_nll = F.gumbel_softmax(tau = tau, dim=1, logits = symbols_logits), F.gumbel_softmax(tau=tau,dim=1,logits=charges_logits), F.gumbel_softmax(tau=tau,dim=1, logits = bonds_logits)

In [ ]:
np.sort(symbols_nll.mean(axis = 0).detach().numpy())

In [ ]:
list(molopt.opt0.parameters())[0]

In [ ]:
molopt.state_dict()

In [ ]:
model_dict = torch.load('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8')

In [ ]:
model_dict["model"]['opt0.weight']

In [ ]:
molopt2 = MolOpt(args)

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt2.load_state_dict(model_dict["model"])

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt3,args3 = load_model('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8', MolOpt, args.device)

In [ ]:
list(molopt3.opt0.parameters())[0]

In [ ]:
fgw_loss(*pred_pack, tau = 1)